In [1]:
using JuMP
using NLopt
using Plots
using Interpolations
using QuadGK
using Cuba

In [2]:
push!(LOAD_PATH, ENV["HOME"]*"/Documents/amplitude_analysis/modules")
# my modules
using isobars
using masses
using DalitzPlotAnalysis

In [10]:
function amplitude(x, p...)
    a = p[1]
    b = p[2]
    c = p[3]
    d = p[4]
    a/(x^2+b^2)+c/(x^2+d^2)
end

amplitude (generic function with 2 methods)

In [12]:
plot(x->amplitude(x,1,1,2,2),-1,10)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 1.25 
 
<polyline clip-path="url(#clip4802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,155.854 40.1541,151.34 41.4965,146.745 42.8389,142.072 55.2513,95.9608 67.6637,49.8985 70.7667,39.8561 73.8698,30.9409 76.9729,23.4377 80.076,17.6111 
 83.1791,13.6829 86.2822,11.811 89.3853,12.0733 92.4884,14.4586 96.218,19.9903 99.9477,28.128 103.677,38.4434 107.407,50.4492 114.866,77.5943 122.326,106.188 
 128.632,129.773 134.939,151.966 141.246,172.399 147.553,190.962 154.404,209.062 161.254,225.162 168.105,239.459 174.956,252.155 187.588,271.99 200.22,288.046 
 213.763,301.984 227.305,313.284 242.996,323.835 258.686,332.294 286.574,343.611 311.029,350.8 337.692,356.678 364.513,361.153 394.167,364.926 422.382,367.701 
 451.719,369.976 476.415,371.531 506.566,373.088 534.908,374.288 580.315,375.813 
 "/>
 
 
 
 
 y1

In [28]:
function ff(x)
    real(quadgk(y->(y+4)/(y-x-1e-3), -50, 60)[1])
end

ff (generic function with 1 method)

In [30]:
ff(1.1)

109.59355952506083

In [31]:
function tr1(y,mod)
    return length([mod(y) for y in 1:10])
end
function tr2(y)
    return length([ff(y) for y in 1:10])
end

tr2 (generic function with 1 method)

In [38]:
using BenchmarkTools

In [39]:
@btime tr1(1.3,ff)

  8.239 s (80000383 allocations: 1.44 GiB)


10

In [40]:
@btime tr2(1.3)

  8.222 s (80000383 allocations: 1.44 GiB)


10

In [42]:
@code_warntype tr1(1.3, ff)

Variables:
  #self# <optimized out>
  y <optimized out>
  mod <optimized out>

Body:
  begin 
      SSAValue(0) = $(Expr(:invoke, MethodInstance for collect(::Base.Generator{UnitRange{Int64},#ff}), :(Base.collect), :($(Expr(:new, Base.Generator{UnitRange{Int64},#ff}, :($(QuoteNode(ff))), :($(Expr(:new, UnitRange{Int64}, 1, :((Base.select_value)((Base.sle_int)(1, 10)::Bool, 10, (Base.sub_int)(1, 1)::Int64)::Int64)))))))))
      return (Base.arraylen)(SSAValue(0))::Int64
  end::Int64


In [43]:
ll = ff

ff (generic function with 1 method)

In [50]:
ll

ff (generic function with 1 method)

In [58]:
function tr4(y)
    return length([hh(y) for y in 1:10])
end

tr4 (generic function with 1 method)

In [48]:
@btime tr2(1.3)

  8.168 s (80000385 allocations: 1.44 GiB)


10

In [63]:
for i=1:3
    i++
end

LoadError: [91msyntax: unexpected "end"[39m

In [405]:
function fit(data,func,stpars) # fits
    # function to minimize
    function rchi2(pars...)
        dd = data[:,1:3]
        sum(begin
                sx = data[i,1];
                dy = data[i,2];
                ey = data[i,3];
                ((dy-func(sx,pars...))/ey)^2
                end for i=1:size(data,1))
    end
    m = Model(solver=NLoptSolver(algorithm=:LD_MMA))
    JuMP.register(m, :rchi2, length(stpars), rchi2, autodiff=true)
    
    jump_pars = [] # to keep parameter references
    for (i,p) in enumerate(stpars)
        @eval push!($jump_pars, @variable($m, $(Symbol("par_"*string(i))), start = $p))
    end
    # string version of the function call
    str = "rchi2(par_1"*prod(",par_"*string(i) for i=2:length(jump_pars))*")"
    
    @eval @NLobjective($m, Min, $(parse(str)))
    print(m)

    # fit!
    status = solve(m)

    println("Objective value: ", getobjectivevalue(m))
    [getvalue(p) for p in jump_pars]
end

fit (generic function with 1 method)

### Comparison of the time-wise efficienty

In [406]:
function test()
    let sv = -1:0.1:2, gunc = (x->x^2+3), func = (x,a,b,c)->a*x^3+b*x^4+c
        data = hcat(sv,gunc.(sv),[0.5 for i in 1:length(sv)])
        pars = fit(data,func,(1,2,4)) # fit
        # plot
        plot(data[:,1],data[:,2],yerr=data[:,3],lw=0, lab = "data")
        plot!(data[:,1],[func(s,pars...) for s in data[:,1]], lab = "fit")
    end
end

test (generic function with 1 method)

In [408]:
@time test()

Min rchi2(par_1, par_2, par_3)
Subject to
 par_1
 par_2
 par_3
Objective value: 12.16777608947337
  0.010143 seconds (21.14 k allocations: 1.303 MiB)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
<polyline clip-path="url(#clip5802)" style="stroke:#009af9; stroke-width:0; stroke-opacity:1; fill:none" points="
 38.1295,262.492 55.6759,275.541 73.2224,287.217 90.7688,297.519 108.315,306.447 125.862,314.002 143.408,320.183 160.955,324.99 178.501,328.424 196.048,330.485 
 213.594,331.172 231.14,330.485 248.687,328.424 266.233,324.99 283.78,320.183 301.326,314.002 318.873,306.447 336.419,297.519 353.966,287.217 371.512,275.541 
 389.059,262.492 406.605,248.069 424.152,232.273 441.698,215.103 459.244,196.559 476.791,176.642 494.337,155.352 511.884,132.687 529.43,108.649 546.977,83.2379 
 564.523,56.4528 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip5802)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.1295,309.15 55.6759,308.267 73.2224,307.278 90.7688,306.289 108.315,305.381 125.862,304.613 143.408,304.02 160.955,303.612 178.501,303.378 196.048,303.284 
 213.594,303.269 231.14,303.252 248.687,303.128 266.233,302.768 283.78,302.02 301.326,300.707 318.873,298.632 336.419,295.571 353.966,291.279 371.512,285.488 
 389.059,277.904 406.605,268.211 424.152,256.071 441.698,241.12 459.244,222.973 476.791,201.221 494.337,175.43 511.884,145.144 529.43,109.884 546.977,69.1476 
 564.523,22.4076 
 "/>
 
 
 
 
 data 
 
 
 
 fit

Straightforward implementation

In [392]:
function my_fit()
    sv = -1:0.1:2; gunc = x->x^2+3; func = (x,a,b,c)->a*x^3+b*x^4+c
    data = hcat(sv,gunc.(sv),[0.5 for i in 1:length(sv)])
    function fit() # fits
        # function to minimize
        function rchi2(a,b,c)
            dd = data[:,1:3]
            sum([begin
                    sx = data[i,1];
                    dy = data[i,2];
                    ey = data[i,3];
                    ((dy-func(sx,a,b,c))/ey)^2
                        end for i=1:size(data,1)])
        end
        m = Model(solver=NLoptSolver(algorithm=:LD_MMA))
        JuMP.register(m, :rchi2, 3, rchi2, autodiff=true)

        @variable(m, p1, start = 1)
        @variable(m, p2, start = 2)
        @variable(m, p3, start = 3)
        
        @NLobjective(m, Min, rchi2(p1,p2,p3))
        print(m)

        # fit!
        status = solve(m)

        println("Objective value: ", getobjectivevalue(m))
        getvalue(p1),getvalue(p2),getvalue(p3)
    end
    pars = fit()
    plot(data[:,1],data[:,2],yerr=data[:,3],lw=0, lab = "data")
    plot!(data[:,1],[func(s,pars...) for s in data[:,1]], lab = "fit")
end

my_fit (generic function with 1 method)

In [394]:
@time my_fit()

Min rchi2(p1, p2, p3)
Subject to
 p1
 p2
 p3
Objective value: 12.167376147780919
  0.006948 seconds (22.77 k allocations: 1.949 MiB)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
<polyline clip-path="url(#clip4202)" style="stroke:#009af9; stroke-width:0; stroke-opacity:1; fill:none" points="
 38.1295,262.492 55.6759,275.541 73.2224,287.217 90.7688,297.519 108.315,306.447 125.862,314.002 143.408,320.183 160.955,324.99 178.501,328.424 196.048,330.485 
 213.594,331.172 231.14,330.485 248.687,328.424 266.233,324.99 283.78,320.183 301.326,314.002 318.873,306.447 336.419,297.519 353.966,287.217 371.512,275.541 
 389.059,262.492 406.605,248.069 424.152,232.273 441.698,215.103 459.244,196.559 476.791,176.642 494.337,155.352 511.884,132.687 529.43,108.649 546.977,83.2379 
 564.523,56.4528 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip4202)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.1295,309.409 55.6759,308.445 73.2224,307.394 90.7688,306.359 108.315,305.418 125.862,304.628 143.408,304.02 160.955,303.604 178.501,303.367 196.048,303.271 
 213.594,303.256 231.14,303.239 248.687,303.114 266.233,302.751 283.78,301.998 301.326,300.679 318.873,298.594 336.419,295.522 353.966,291.218 371.512,285.413 
 389.059,277.816 406.605,268.112 424.152,255.963 441.698,241.009 459.244,222.865 476.791,201.124 494.337,175.355 511.884,145.106 529.43,109.9 546.977,69.2357 
 564.523,22.5916 
 "/>
 
 
 
 
 data 
 
 
 
 fit

## Minimization